In [15]:
!nvidia-smi

Sat May  8 09:12:31 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [16]:
!pip install -q kaggle

In [17]:
from google.colab import files
files.upload()          

Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"aarushverma","key":"a556b53644acf230d4623a725d843ac2"}'}

In [18]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [19]:
!kaggle competitions download -c nlp-getting-started

test.csv: Skipping, found more recently modified local copy (use --force to force download)
sample_submission.csv: Skipping, found more recently modified local copy (use --force to force download)
train.csv: Skipping, found more recently modified local copy (use --force to force download)


In [20]:
# nltk
import nltk
from nltk.corpus import stopwords
from  nltk.stem import SnowballStemmer

# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

#Utility
import numpy as np
import pandas as pd
import re

# Word2vec
import gensim

# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, Embedding, Flatten, Conv1D, MaxPooling1D, LSTM
from keras import utils
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

# SENTIMENT
POSITIVE = "POSITIVE"
NEGATIVE = "NEGATIVE"
NEUTRAL = "NEUTRAL"
SENTIMENT_THRESHOLDS = (0.4, 0.7)

In [21]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [88]:
# DATASET
TRAIN_SIZE = 0.8

# KERAS
SEQUENCE_LENGTH = 50
EPOCHS = 50
BATCH_SIZE = 32

# WORD2VEC 
W2V_SIZE = 300
W2V_WINDOW = 7
W2V_EPOCH = 32
W2V_MIN_COUNT = 3

# TEXT CLEANING
TEXT_CLEANING_RE = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"

In [23]:
df = pd.read_csv('/content/train.csv',
                 engine='python')
df.head(5)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [24]:
stop_words = stopwords.words("english")
stemmer = SnowballStemmer("english")

In [25]:
def preprocess(text, stem=False):
    text = re.sub(TEXT_CLEANING_RE, ' ', str(text).lower()).strip()
    tokens = []
    for token in text.split():
        if token not in stop_words:
            if stem:
                tokens.append(stemmer.stem(token))
            else:
                tokens.append(token)
    return " ".join(tokens)

In [26]:
df.text = df.text.apply(lambda z: preprocess(z))

In [75]:
df_test = pd.read_csv('/content/test.csv',
                 engine='python')
df_test.text = df_test.text.apply(lambda z: preprocess(z))
df_train = df
print("TRAIN size:", len(df_train))
print("TEST size:", len(df_test))

TRAIN size: 7613
TEST size: 3263


In [28]:
df_train.head(5)

,id,keyword,location,text,target
0,1,NaN,NaN,deeds reason earthquake may allah forgive us,1
1,4,NaN,NaN,forest fire near la ronge sask canada,1
2,5,NaN,NaN,residents asked shelter place notified officer...,1
3,6,NaN,NaN,13 000 people receive wildfires evacuation ord...,1
4,7,NaN,NaN,got sent photo ruby alaska smoke wildfires pou...,1


In [29]:
documents = [_text.split() for _text in df_train.text] 

In [30]:
w2v_model = gensim.models.word2vec.Word2Vec(size=W2V_SIZE, 
                                            window=W2V_WINDOW, 
                                            min_count=W2V_MIN_COUNT, 
                                            workers=8)

In [31]:
w2v_model.build_vocab(documents)

In [32]:
words = w2v_model.wv.vocab.keys()
vocab_size = len(words)
print("Vocab size", vocab_size)

Vocab size 4346


In [33]:
words

dict_keys(['reason', 'earthquake', 'may', 'allah', 'us', 'forest', 'fire', 'near', 'la', 'canada', 'residents', 'asked', 'shelter', 'place', 'officers', 'evacuation', 'orders', 'expected', '13', '000', 'people', 'wildfires', 'california', 'got', 'sent', 'photo', 'alaska', 'smoke', 'school', 'rockyfire', 'update', 'hwy', '20', 'closed', 'due', 'lake', 'county', 'flood', 'disaster', 'heavy', 'rain', 'causes', 'flash', 'flooding', 'streets', 'colorado', 'springs', 'areas', 'top', 'hill', 'see', 'emergency', 'happening', 'building', 'across', 'street', 'afraid', 'tornado', 'coming', 'area', 'three', 'died', 'heat', 'wave', 'far', 'haha', 'south', 'tampa', 'getting', 'flooded', 'hah', 'wait', 'second', 'live', 'gonna', 'raining', 'florida', '18', '19', 'days', 'lost', 'count', 'myanmar', 'arrived', 'damage', 'bus', '80', 'multi', 'car', 'crash', 'breaking', 'man', 'love', 'summer', 'lovely', 'fast', 'ridiculous', 'london', 'cool', 'wonderful', 'day', 'way', 'eat', 'shit', 'nyc', 'last', 'we

In [34]:
w2v_model.train(documents, total_examples=len(documents), epochs=W2V_EPOCH)

(1807503, 2259392)

In [35]:
w2v_model.most_similar("forest")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('national', 0.9120702147483826),
 ('service', 0.9074880480766296),
 ('wild', 0.8944728374481201),
 ('budget', 0.8941150903701782),
 ('fires', 0.8912601470947266),
 ('spending', 0.8849135637283325),
 ('acres', 0.8813414573669434),
 ('fighting', 0.8727735877037048),
 ('bush', 0.8720462918281555),
 ('contained', 0.8718241453170776)]

In [36]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df_train.text)

vocab_size = len(tokenizer.word_index) + 1
print("Total words", vocab_size)

Total words 15622


In [76]:
x_train = pad_sequences(tokenizer.texts_to_sequences(df_train.text), maxlen=SEQUENCE_LENGTH)
x_test = pad_sequences(tokenizer.texts_to_sequences(df_test.text), maxlen=SEQUENCE_LENGTH)

In [62]:
labels = df_train.target.unique().tolist()
labels

[1, 0]

In [63]:
y_train = np.array(df_train.target.tolist())
y_train = y_train.reshape(-1,1)
print("y_train",y_train.shape)

y_train (7613, 1)


In [64]:
print("x_train", x_train.shape)
print("y_train", y_train.shape)
print()
print("x_test", x_test.shape)

x_train (7613, 50)
y_train (7613, 1)

x_test (3263, 50)


In [65]:
y_train[:10]

array([[1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1]])

In [66]:
embedding_matrix = np.zeros([vocab_size, W2V_SIZE])
for word,idx in tokenizer.word_index.items():
    if word in w2v_model.wv:
        embedding_matrix[idx] = w2v_model.wv[word]
print(embedding_matrix.shape)

(15622, 300)


In [89]:
embedding_layer = Embedding(vocab_size, W2V_SIZE, weights=[embedding_matrix], input_length=SEQUENCE_LENGTH, trainable=False)

In [90]:
model = Sequential()
model.add(embedding_layer)
#model.add(Dropout(0.5))
model.add(LSTM(100, dropout=0.1, recurrent_dropout=0.1))
model.add(Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 50, 300)           4686600   
_________________________________________________________________
lstm_5 (LSTM)                (None, 100)               160400    
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 101       
Total params: 4,847,101
Trainable params: 160,501
Non-trainable params: 4,686,600
_________________________________________________________________


In [91]:
model.compile(loss='binary_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])

In [92]:
callbacks = [ ReduceLROnPlateau(monitor='val_loss', patience=5, cooldown=0),
              EarlyStopping(monitor='val_acc', min_delta=1e-4, patience=5)]

In [93]:
history = model.fit(x_train, y_train,
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    verbose=1)#,
                    #callbacks=callbacks)

Epoch 1/50
238/238 [==============================] - 39s 156ms/step - loss: 0.5301 - accuracy: 0.7525
Epoch 2/50
238/238 [==============================] - 37s 155ms/step - loss: 0.4649 - accuracy: 0.7898
Epoch 3/50
238/238 [==============================] - 37s 156ms/step - loss: 0.4484 - accuracy: 0.7947
Epoch 4/50
238/238 [==============================] - 37s 156ms/step - loss: 0.4326 - accuracy: 0.8047
Epoch 5/50
238/238 [==============================] - 37s 155ms/step - loss: 0.4239 - accuracy: 0.8095
Epoch 6/50
238/238 [==============================] - 37s 157ms/step - loss: 0.4095 - accuracy: 0.8130
Epoch 7/50
238/238 [==============================] - 37s 156ms/step - loss: 0.3987 - accuracy: 0.8237
Epoch 8/50
238/238 [==============================] - 37s 155ms/step - loss: 0.3842 - accuracy: 0.8287
Epoch 9/50
238/238 [==============================] - 37s 158ms/step - loss: 0.3716 - accuracy: 0.8417
Epoch 10/50
238/238 [==============================] - 37s 156ms/step - l

In [94]:
score = model.evaluate(x_train, y_train, batch_size=BATCH_SIZE)
print()
print("ACCURACY:",score[1])
print("LOSS:",score[0])

238/238 [==============================] - 3s 12ms/step - loss: 0.0650 - accuracy: 0.9765

ACCURACY: 0.9764875769615173
LOSS: 0.06504594534635544


In [95]:
pred = model.predict_classes(x_test)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [96]:
import csv
rowlist = []
rowlist.append(['id','target'])
for i in range(len(pred)):
    rowlist.append([df_test.id[i], pred[i][0]])
with open('submission.csv', 'w', newline = '') as file:
    writer = csv.writer(file)
    writer.writerows(rowlist)